# Proyecto - Asistente de IA para aerolíneas

## GEMINI 1.5 - uso de herramientas

* https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
import gradio as gr

In [2]:
import google.generativeai as genai

# Inicialización

load_dotenv()

google_api_key = os.getenv('GOOGLE_API_KEY')

if google_api_key:
    print(f"Google API Key existe y empieza por {google_api_key[:8]}")
else:
    print("Google API Key Sin Configurar")

genai.configure(api_key= google_api_key)

Google API Key existe y empieza por AIzaSyCw


### Declaramos las herramientas

In [3]:
# https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb

ticket_prices = {"londres": "799€", "parís": "899€", "tokyo": "1400€", "berlín": "499€"}

def contar(a: str):
    """returns result"""
    result= len(a)
    return result
    
def get_ticket_price(destination_city: str):
    """returns result"""
    print(f"Se solicitó la herramienta get_ticket_price para {destination_city}")
    city = destination_city.lower()
    result = ticket_prices.get(city, "Unknown")
    return result

def get_flight_price(city: str):
    """returns result"""
    print(f"Se solicitó la herramienta get_flight_price para {city}")
    city = city.lower()
    result = ticket_prices.get(city, "Unknown")
    return result


### Personalizamos el comportamiento del modelo y referimos las herramientas

In [4]:
system_message = "Eres un asistente útil para una aerolínea llamada FlightAI. "
system_message += "Da respuestas breves y corteses, de no más de una oración. "
system_message += "Se siempre preciso. Si no sabes la respuesta, dilo."

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", 
    system_instruction=system_message,
    tools=[contar, get_ticket_price, get_flight_price]
)

print (model)

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x000001ECA98FDAD0>,
    system_instruction='Eres un asistente útil para una aerolínea llamada FlightAI. Da respuestas breves y corteses, de no más de una oración. Se siempre preciso. Si no sabes la respuesta, dilo.',
    cached_content=None
)


In [8]:
chat= model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message("¿Cuál es el precio de un ticket a Londres?")
response.text

Se solicitó la herramienta get_ticket_price para Londres


'El precio de un billete a Londres es de 799€.\n'

In [9]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "El precio de un billete a Londres es de 799\u20ac.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.012815107591450214
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 98,
        "candidates_token_count": 16,
        "total_token_count": 114
      }
    }),
)


In [10]:
chat = model.start_chat(history=[], enable_automatic_function_calling=True)

def chatter(prompt, chat_history):
    print(prompt)

    if prompt :
        response = chat.send_message(prompt)

    output = response.candidates[0].content.parts[0].text
    return output

In [11]:
gr.ChatInterface(fn=chatter, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Hola
Me gustaría volar a Londres, ciudad que me encanta visitar
Cuánto cuesta el vuelo desde Madrid?
el proximo lunes
Se solicitó la herramienta get_flight_price para Madrid
Y un vuelo a Berlín?
Se solicitó la herramienta get_flight_price para Berlín
